<h3>predicting future sales using basic machine learning</h3>

In [141]:
import numpy as np
import pandas as pd
import os

In [142]:
from sklearn import *

In [143]:
import nltk,datetime

In [144]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

In [145]:
test=pd.read_csv("C:\Windows\Lenovo\\test.csv")
item=pd.read_csv("C:\Windows\Lenovo\\items.csv")
item_cat=pd.read_csv("C:\Windows\Lenovo\\item_categories.csv")
shop=pd.read_csv("C:\Windows\Lenovo\\shops.csv")
sample_sub=pd.read_csv("C:\Windows\Lenovo\\sample_submission.csv")
train=pd.read_csv("C:\Windows\Lenovo\\sales_train.csv")

In [146]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [147]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [148]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [149]:
sample_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [150]:
item.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [151]:
shop.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [152]:
train['date']=pd.to_datetime(train['date'],format='%d.%m.%Y')
train['month']=train['date'].dt.month
train['year']=train['date'].dt.year
train=train.drop('date',axis=1)
train=train.drop('item_price',axis=1)

In [153]:
train=train.groupby([c for c in train.columns if c not in['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()

In [154]:
train=train.rename(columns={'item_cnt_day':'item_cnt_month'})

In [155]:
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month
0,0,0,32,1,2013,6.0
1,0,0,33,1,2013,3.0
2,0,0,35,1,2013,1.0
3,0,0,43,1,2013,1.0
4,0,0,51,1,2013,2.0


In [156]:
shop_item_mean=train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_mean=shop_item_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})

In [157]:
train=pd.merge(train,shop_item_mean,how='left', on=['shop_id','item_id'])

In [158]:
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean
0,0,0,32,1,2013,6.0,8.0
1,0,0,33,1,2013,3.0,3.0
2,0,0,35,1,2013,1.0,7.5
3,0,0,43,1,2013,1.0,1.0
4,0,0,51,1,2013,2.0,2.5


In [159]:
shop_prev_month=train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]

In [160]:
shop_prev_month=shop_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})

In [161]:
shop_prev_month.head()

,shop_id,item_id,item_cnt_prev_month
1577593,2,31,1.0
1577594,2,486,3.0
1577595,2,787,1.0
1577596,2,794,1.0
1577597,2,968,1.0


In [162]:
train=pd.merge(train,shop_prev_month,how='left',on=['shop_id','item_id']).fillna(0.)

In [163]:
train=pd.merge(train,item,how='left',on=['item_id'])

In [164]:
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id
0,0,0,32,1,2013,6.0,8.0,0.0,1+1,40
1,0,0,33,1,2013,3.0,3.0,0.0,1+1 (BD),37
2,0,0,35,1,2013,1.0,7.5,0.0,10 ЛЕТ СПУСТЯ,40
3,0,0,43,1,2013,1.0,1.0,0.0,100 МИЛЛИОНОВ ЕВРО,40
4,0,0,51,1,2013,2.0,2.5,0.0,100 лучших произведений классики (mp3-CD) (Dig...,57


In [165]:
train=pd.merge(train,item_cat,how='left',on=['item_category_id'])

In [166]:
train=pd.merge(train,shop,how='left',on='shop_id')

In [167]:
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,1,2013,6.0,8.0,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,1,2013,3.0,3.0,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,1,2013,1.0,7.5,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,1,2013,1.0,1.0,0.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,1,2013,2.0,2.5,0.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"


In [168]:
test['month']=11
test['year']=2015
test['date_block_num']=34

In [169]:
test=pd.merge(test,shop_item_mean,how='left', on=['shop_id','item_id']).fillna(0.)

In [170]:
test=pd.merge(test,shop_prev_month,how='left',on=['shop_id','item_id']).fillna(0.)

In [171]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month
0,0,5,5037,11,2015,34,1.444444,0.0
1,1,5,5320,11,2015,34,0.000000,0.0
2,2,5,5233,11,2015,34,2.000000,1.0
3,3,5,5232,11,2015,34,1.000000,0.0
4,4,5,5268,11,2015,34,0.000000,0.0


In [172]:
test=pd.merge(test,shop,how='left',on='shop_id')

In [173]:
test=pd.merge(test,item,how='left',on=['item_id'])

In [174]:
test=pd.merge(test,item_cat,how='left',on=['item_category_id'])

In [175]:
test['item_cnt_month']=0

In [176]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,shop_name,item_name,item_category_id,item_category_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,"Вологда ТРЦ ""Мармелад""","NHL 15 [PS3, русские субтитры]",19,Игры - PS3,0
1,1,5,5320,11,2015,34,0.000000,0.0,"Вологда ТРЦ ""Мармелад""",ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,0
2,2,5,5233,11,2015,34,2.000000,1.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,0
3,3,5,5232,11,2015,34,1.000000,0.0,"Вологда ТРЦ ""Мармелад""","Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,0
4,4,5,5268,11,2015,34,0.000000,0.0,"Вологда ТРЦ ""Мармелад""","Need for Speed [PS4, русская версия]",20,Игры - PS4,0


In [177]:
for c in ['shop_name','item_name','item_category_name']:
    lbl=preprocessing.LabelEncoder()
    lbl.fit(list(train[c].unique())+list(test[c].unique()))
    train[c]=lbl.transform(train[c].astype(str))
    test[c]=lbl.transform(test[c].astype(str))
    print(c)

shop_name
item_name
item_category_name


In [178]:
hide_toggle(for_next=True)

In [179]:
l=[c for c in train.columns if c not in ['item_cnt_month']]
x1=train[train['date_block_num']<33]
y1=np.log1p(x1['item_cnt_month'].clip(0.,20.))
x1=x1[col]
x2=train[train['date_block_num']==33]
y2=np.log1p(x2['item_cnt_month'].clip(0.,20.))
x2=x2[col]
reg=ensemble.ExtraTreesRegressor(n_estimators=30,n_jobs=-1,max_depth=15,random_state=18)
reg.fit(x1,y1)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=15,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=-1,
          oob_score=False, random_state=18, verbose=0, warm_start=False)

In [180]:
print('RMSE value is:',np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.))))

RMSE value is: 0.26922329004087775


In [181]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,shop_name,item_name,item_category_id,item_category_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,5,5037,19,19,0
1,1,5,5320,11,2015,34,0.000000,0.0,5,5320,55,55,0
2,2,5,5233,11,2015,34,2.000000,1.0,5,5233,19,19,0
3,3,5,5232,11,2015,34,1.000000,0.0,5,5232,23,23,0
4,4,5,5268,11,2015,34,0.000000,0.0,5,5268,20,20,0


In [182]:
reg.fit(train[col],train['item_cnt_month'].clip(0.,20.))
test['item_cnt_month']=reg.predict(test[col]).clip(0.,20.)
test[['ID','item_cnt_month']].to_csv("submit.csv",index=False)

In [183]:
test['item_cnt_month']=np.expm1(test['item_cnt_month'])
test[['ID','item_cnt_month']].to_csv("final_submit.csv",index=False)